In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [3]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
using Test
using ForwardDiff
using SpecialFunctions
using Polynomials
using BenchmarkTools
using TransportMap
using QuadGK
using FastGaussQuadrature
using AdaptiveTransportMap: derivative, vander, scale!, evaluate
using DiffResults
using Distributions
using Random
using TensorOperations
using Cubature
using LoopVectorization
using Optim

┌ Info: Precompiling Optim [429524aa-4258-5aef-a3af-852621145aeb]
└ @ Base loading.jl:1260


In [7]:
rosenbrock(x) =  (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
@time result = optimize(rosenbrock, zeros(2), LBFGS())

  0.015897 seconds (21.64 k allocations: 1.055 MiB)


 * Status: success

 * Candidate solution
    Final objective value:     5.378388e-17

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 4.54e-11 ≰ 0.0e+00
    |x - x'|/|x'|          = 4.54e-11 ≰ 0.0e+00
    |f(x) - f(x')|         = 2.85e-19 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 5.30e-03 ≰ 0.0e+00
    |g(x)|                 = 9.88e-14 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    24
    f(x) calls:    67
    ∇f(x) calls:   67
